In [1]:
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

# Bibliographie trouvée

** en bio : **

* Drawing explicit phylogenetic networks and their integration into SplitsTree
https://link.springer.com/article/10.1186/1471-2148-8-22

**Majeur :**
 * NEW  COMMON  ANCESTOR  PROBLEMS  IN  TREE AND  DIRECTED  ACYCLIC  GRAPHS
         ->  the  “lowest single common ancestor” (LSCA) 
     https://pdfs.semanticscholar.org/71a9/b5dbfde43fe3b40d91e8ca24a49e7638b4f7.pdf

        Fischer, Johannes, and Daniel H. Huson. "New common ancestor problems in trees and directed acyclic graphs." Information processing letters 110.8-9 (2010): 331-335.

 et    page 307 : Summarizing Multiple Gene Trees Using Cluster Networks  
`ftp://nozdr.ru/biblio/kolxo3/Cs/CsLn/Algorithms%20in%20Bioinformatics,%208%20conf.,%20WABI%202008(LNCS5251,%20Springer,%202008)(ISBN%209783540873600)(406s).pdf#page=307')`
 
 
** Stackoverflow **
* Lowest single common ancestor in a Directed Acyclic Graph? : https://cs.stackexchange.com/q/72538
* https://stackoverflow.com/q/14865081/8069403

 ** Category of citation ** is one of {CH2 = Chapter 2; SUP = Supplementary search report ; ISR = International search report ; SEA = Search report; APP = Applicant; EXA = Examiner; OPP = Opposition; 115 = article 115; PRS = Pre-grant pre-search; APL = Appealed; FOP = Filed opposition}, Type is one of {A = technological background; D = document cited in application; E = earlier patent document; 1 = document cited for other reasons; O = Non-written disclosure; P = Intermediate document; T = theory or principle; X = relevant if taken alone; Y = relevant if combined with other documents}

# Graph, test de NetworkX

le site https://networkx.github.io/ et  la  [documentation](https://networkx.github.io/documentation/stable/index.html).


## Création du graph

In [2]:
import pickle
data = pickle.load( open( "../web/data/patent_infos.pickle", "rb" ) )

print(len(data))

379


In [3]:
import networkx as nx

In [4]:
G = nx.DiGraph()

In [5]:
# add nodes

patents = list( data.keys() )
G.add_nodes_from( patents )

G.number_of_nodes()

379

In [6]:
# add edges

for A, patent in data.items():
    dateA = patent['year'], patent['month'], patent['day']
    
    for B in patent['cited']:
        patentB = data[B]
        dateB = patentB['year'], patentB['month'], patentB['day']
        
        if dateB < dateA:  # hack pour retirer les cycles... date pub. VS date priority ??
            G.add_edge( B, A ) # la fleche est dans le sens du temps
        
G.number_of_edges()

1149

In [7]:
nx.is_directed_acyclic_graph( G )

True

In [8]:
nx.is_weakly_connected( G )

False

In [9]:
nx.is_strongly_connected( G )

False

# Nombre de composante

In [10]:
components = list( nx.weakly_connected_components( G ))
print(len(components))

68


In [11]:
print( [len(c) for c in sorted(nx.weakly_connected_components(G), key=len, reverse=True)] )

[312, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [12]:
# Keep only the largest component :
G = max(nx.weakly_connected_component_subgraphs(G), key=len)

G.number_of_nodes(), G.number_of_edges()

(312, 1149)

In [13]:
nx.is_directed_acyclic_graph( G )

True

# feuilles

In [ ]:
leafs = [ n[0] for n in G.out_degree if n[1] == 0 ]
print(len( leafs ))
print(leafs)

In [ ]:
allancestors = { L:nx.ancestors( G, L ) for L in leafs }

In [ ]:
# graph
size_ancestors = sorted( [ len( A ) for A in allancestors.values() ] )

plt.step( range(len(size_ancestors)), size_ancestors )
plt.xlabel('sorted leaf nodes')
plt.ylabel('size of Ancestors( leaf )');

In [ ]:
# brevet avec le plus d'ancetres... (ce sont des brevets qui cite beaucoup d'autre brevet...)
print( (sorted( [ (k, len( A )) for k, A in allancestors.items() ], key=lambda x:x[1] )[-3:] ) )

In [ ]:
list(allancestors.items())[0]

In [ ]:
def dico_append( dico, key, value ):
    if key in dico:
        dico[key].append( value )
    else:
        dico[key] = [value]

In [ ]:
branchs_from_node = {}

for leafnode, ancestorsnodes in allancestors.items():
    
    for node in ancestorsnodes:
        dico_append( branchs_from_node, node, leafnode )
        
print( len( branchs_from_node ))

In [ ]:
254+58

In [ ]:
G.number_of_nodes()

In [ ]:
branches = { tuple( sorted(b) ) for b in branchs_from_node.values() }
print(len(branches))

In [ ]:
nodes_in_branches = {}

for node, branche in branchs_from_node.items():
    
    branche_hashable = tuple( sorted(branche)) 
    dico_append( nodes_in_branches, branche_hashable, node )
    
print(len(nodes_in_branches))

In [ ]:
branches_stats = sorted( [ (k, len(v)) for k, v in nodes_in_branches.items() ], key=lambda x:x[1], reverse=True )

In [ ]:
plt.plot( [ x[1] for x in branches_stats] );
plt.xlabel('branches sorted')
plt.ylabel('number of nodes')

print( sum( [ x[1] for x in branches_stats] ) )

In [ ]:
plt.plot( sorted( [ len(b) for b in branches ] ), 'r' );
plt.xlabel('branches sorted')
plt.ylabel('number leaf nodes in the branch def.')


In [ ]:
branches_stats2 =  [ (len(k), len(v)) for k, v in nodes_in_branches.items() ]

In [ ]:
plt.plot( *list( zip( *branches_stats2 ) ), '.r' , alpha=0.5);

plt.xlabel('number leaf nodes in the branch def.')
plt.ylabel('number of nodes in the branch')

In [ ]:
for b in sorted( branches, reverse=True ) :
    print( ', '.join( b ) )
    print('')

### Knowledge

l'idée est de supprimer les citations redondantes. Souvent, un article ancien est cité, alors qu'il est déjà cité par les publications citées. 

**definition n°0 :** pour tout article cité, si il existe un autre chemin entre lui et son article directement parent, alors on supprime le lien direct.



In [14]:
def knowledge( G ):
    Gk = G.copy()

    for a in G.nodes():
        for cited in G.predecessors(a):
            allpaths = list( nx.all_simple_paths( G, cited, a ) )
            if(len(allpaths)) > 1:
                # supprime le lien
                Gk.remove_edge( cited, a )
                
    return Gk

In [24]:
G = knowledge( G )

In [25]:
G.number_of_edges()

741

# evolution tree

In [16]:
# add virtual roor
roots = [ n[0] for n in G.in_degree if n[1] == 0 ]
print(len( roots ))
print(roots)

88
['US20030140498', 'US1363164', 'US3712311', 'US1436010', 'US1499673', 'US770032', 'US766859', 'US1836557', 'US20020092172', 'US1702137', 'US650869', 'US794234', 'US138061', 'US20030094183', 'US2028558', 'US3089239', 'US796389', 'US1394727', 'US797576', 'US6155270', 'US2532370', 'US1001727', 'US1721415', 'US869946', 'US2146132', 'US1430779', 'US20020134395', 'USRE26344', 'US183256', 'US1855063', 'US2480797', 'US768083', 'US898808', 'US327065', 'US20040123875', 'US843383', 'US512277', 'US806037', 'US20160066542', 'US762725', 'US2960766', 'US695342', 'US1727239', 'US853832', 'US20110290080', 'US1085569', 'US2540782', 'US954325', 'US4328819', 'US3555675', 'US244891', 'US2704398', 'US838755', 'US3832771', 'US1976067', 'US970067', 'US20070017540', 'US1447991', 'US1841847', 'US3680210', 'US846565', 'US857790', 'US741709', 'US1135987', 'US1081896', 'US1426696', 'US20020178585', 'US1125577', 'US523708', 'US4739552', 'US860975', 'US450703', 'US775568', 'US1290380', 'US1089019', 'US1849592', '

In [17]:
G.add_node( 0 )

In [18]:
for r in roots:
    G.add_edge( 0, r )

In [19]:
allpaths = {}
def followthepath( source, previouspath=[] ):

    path =  list( previouspath )    
    path.append( source )

    if source in allpaths:
        allpaths[source].append( path )
    else:
        allpaths[source] = [ path ]
    
    for n in  G.successors(source):
        followthepath( n, path )
        
followthepath(0)

LSA = {}
n_apparition = lambda v, pathsToNode : sum( [ 1 for path in pathsToNode if v in path ] )

for v in nx.nodes(G):
    pathsToV = allpaths[ v ]
    
    for node in nx.dfs_postorder_nodes(G):
    
        if len( pathsToV ) == n_apparition( node, pathsToV ) and v!=node:
            LSA[ v ] = node
            break
            
print(len(LSA))

312


In [20]:
T = nx.Graph()

In [21]:
T.add_nodes_from( LSA.keys() )

In [22]:
T.add_edges_from( LSA.items() )

In [23]:
nx.write_graphml(T, 'evolutiontree02.graphml' )

In [26]:
nx.write_graphml(G, 'knowkedgegraph.graphml' )